In [2]:
import sys
sys.path.append('../../Share/')
sys.path.append('../../V5 Self Supervised Learning')

import pandas as pd
import numpy as np
from sklearn.utils import resample
import matplotlib.pyplot as plt
import baseline, config, self_supervised_v1

import warnings
warnings.filterwarnings('ignore')

def evaluate_model(model, data, labels):
    return model.evaluate(data, labels, verbose=0)[1]


def down_sample(X, y):
    X = np.array(X)
    y = np.array(y)

    # 클래스별 최소 개수
    unique_classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()

    X_balanced, y_balanced = [], []

    for cls in unique_classes:
        idx = np.where(y == cls)[0]  # 해당 클래스 인덱스
        down_idx = resample(idx, replace=False, n_samples=min_count, random_state=42)
        X_balanced.append(X[down_idx])
        y_balanced.append(y[down_idx])

    # 합치기
    X_balanced = np.concatenate(X_balanced, axis=0)
    y_balanced = np.concatenate(y_balanced, axis=0)

    return X_balanced, y_balanced



trainer_Minjeong = baseline.ModelTrainer(config, subject="Minjeong")
trainer_Carlson = baseline.ModelTrainer(config, subject="Carlson")
trainer_Harold = baseline.ModelTrainer(config, subject="Harold")
trainer_Hunmin = baseline.ModelTrainer(config, subject="Hunmin")
trainer_Brian = baseline.ModelTrainer(config, subject="Brian")
trainer_Xianyu = baseline.ModelTrainer(config, subject="Xianyu")

In [1]:
def generate_pseudo_label(model, x):
    y_pred = model.predict(x, verbose=0)
    return np.argmax(y_pred, axis=1)

def online_update(model, new_sample, pseudo_label, X_test, y_test):
    #model.train_on_batch(new_sample, pseudo_label)

    history = model.fit(new_sample, pseudo_label, batch_size=256, epochs=50, validation_data=(X_test, y_test), verbose=0)
    return np.max(history.history['val_accuracy']), model

def evaluate_model(model, data, labels):
    return model.evaluate(data, labels, verbose=0)[1]

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import meta

trainers_unseen = {
    "H": trainer_Hunmin,
    "C": trainer_Carlson,
    "B": trainer_Brian,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}
#Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
#Unseen_subject_acc_dict = {"H": [], "C": [], "B": [], "X": [], "M": []}
n_classes = 6

In [ ]:
Harold_CM = []

In [ ]:
model = load_model('./model_K10_H2.h5')
model.compile(optimizer=Adam(learning_rate=0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

cm_total = np.zeros((n_classes, n_classes), dtype=np.int32)
best_acc = -1
best_cm = None

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:], meta_iters=5, meta_step_size=1)
    acc, y_pred = Meta.train_for_heatmap(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(f"Run {k} | Acc: {acc:.4f}")
    Acc_lst.append(acc)

    if acc > best_acc:  # 현재까지 최고 acc면 confusion matrix 갱신
        best_acc = acc
        best_cm = confusion_matrix(y_TEST, y_pred, labels=np.arange(n_classes))

# 최종 confusion matrix (best run)
if best_cm is not None:
    cm_total += best_cm
    cm_norm = best_cm.astype('float') / best_cm.sum(axis=1)[:, np.newaxis]  # row-normalized
    Harold_CM.append(cm_total)

In [ ]:
import seaborn as sns
plt.figure(figsize=(8,6))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=np.arange(n_classes),
            yticklabels=np.arange(n_classes))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Average Confusion Matrix (Unseen Subjects)")
plt.show()

In [ ]:
model = load_model('./model_K10_H2.h5')
model.compile(optimizer=Adam(learning_rate=0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

# Hunmin

In [4]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-15-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-08-01-v1/E9AD0E7DCC2B/
0.5416666865348816 0.4728682041168213
Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
0.6000000238418579 0.4674418568611145
Returning K-th session data: Exp_2025-08-02-v1/E9AD0E7DCC2B/
0.5583333373069763 0.5100775361061096
Returning K-th session data: Exp_2025-08-02-v2/E9AD0E7DCC2B/
0.5833333134651184 0.5294573903083801
Returning K-th session data: Exp_2025-08-09-v1/E9AD0E7DCC2B/
0.5833333134651184 0.5612403154373169
Returning K-th session data: Exp_2025-08-09-v2/E9AD0E7DCC2B/
0.6833333373069763 0.6007751822471619
Returning K-th session data: Exp_2025-08-10-v1/E9AD0E7DCC2B/
0.6166666746139526 0.6186046600341797
Returning K-th session data: Exp_2025-08-10-v2/E9AD0E7DCC2B/
0.625 0.6201550364494324
Returning K-th session data: Exp_2025-08-11-v1/E9AD0E7DCC2B/
0.6666666865348816 0.5906976461410522
Returning K-th session data: Exp_2025-08-11-v2/E9AD0E7DCC2B/


In [ ]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/


In [ ]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

# Xianyu

In [ ]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

# Brian

In [ ]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

# Carlson

In [ ]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

# Minjeong

In [ ]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)

In [ ]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst, Acc_after_self_learning = [],[]

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )
    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)

    pseudo_labels = generate_pseudo_label(model, X_train)
    acc2, model = online_update(model, X_train, pseudo_labels, X_TEST, y_TEST)
    Acc_after_self_learning.append(acc2)
    print(acc, acc2)
    Acc_lst.append(acc)